In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('data/礼物长尾词_1606095511.csv')

C:\Users\wyf35\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df = df.sort_values(by='长尾词数量',ascending=False)

In [7]:
df = df.rename(columns={'关键词':'word', '长尾词数量':'lenword_count','PC日检索量(VIP特权数据)':'searched_day'})

In [8]:
df.head(20)

,word,lenword_count,搜索结果,SEM点击价格(SVIP特权数据),流量特点(SVIP特权数据),流量指数(VIP特权数据),移动指数(VIP特权数据),360指数(VIP特权数据),竞价公司数量(VIP特权数据),searched_day,移动日检索量(VIP特权数据),竞价竞争激烈程度(VIP特权数据)
939,《礼物》,4466347.0,47900000,NaN,NaN,0,0,0,-,3,9,3
88289,礼物(),4466347.0,26600000,NaN,NaN,未收录,未收录,0,-,-,-,-
938,《礼物,4466347.0,1500000,NaN,NaN,0,0,0,-,-,-,11
88287,《礼物》礼物,4466347.0,26700000,NaN,NaN,未收录,未收录,0,-,-,-,-
940,()礼物,4466347.0,17500000,NaN,NaN,0,0,0,-,-,-,-
88288,《 礼物 》,4466347.0,1980000,NaN,NaN,未收录,未收录,未收录,-,-,-,-
937,礼物礼物,4466347.0,367000,NaN,NaN,0,0,0,-,1,4,2
3,礼物,4466347.0,100000000,NaN,NaN,1420,968,18704,72,761,1232,1
88290,() 的礼物,1354732.0,100000000,NaN,NaN,未收录,未收录,未收录,-,-,-,-
88291,()()的礼物,1354698.0,26700000,NaN,NaN,未收录,未收录,0,-,-,-,-


#### 数据预清洗
* 基于PC日检索量(VIP特权数据)来进行初步清洗，筛选出有有效值的数据
* word清洗：去掉word中的特殊字符，并去重

In [62]:
used_df = df[['word','lenword_count','searched_day']]

used_df = used_df[used_df['searched_day'] != '-']

In [63]:
used_df.head(20)

,word,lenword_count,searched_day
939,《礼物》,4466347.0,3
937,礼物礼物,4466347.0,1
3,礼物,4466347.0,761
941,的礼物,1339503.0,2
942,礼物的,1339503.0,1
236,什么礼物,1133560.0,4
946,礼物送什么,613979.0,1
188,送什么礼物,613979.0,11
88292,生日 礼物,562027.0,1
947,礼物生日,562027.0,1


In [64]:
# 去掉word中的特殊字符
def remove_col_str(df,col):

    df[col].replace('\n\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a', '', regex=True, inplace=True)
    df[col].replace(' &#.*' , '', regex=True, inplace=True)
    df[col].replace('《|》| ', '', regex=True, inplace=True)


remove_col_str(used_df,'word')

# 去重
used_df = used_df.sort_values(by='searched_day',ascending=False).drop_duplicates(['word'],keep='first')
used_df = used_df.sort_values(by='lenword_count',ascending=False)

In [66]:
used_df.head(20)

,word,lenword_count,searched_day
937,礼物礼物,4466347.0,1
3,礼物,4466347.0,761
941,的礼物,1339503.0,2
942,礼物的,1339503.0,1
236,什么礼物,1133560.0,4
188,送什么礼物,613979.0,11
946,礼物送什么,613979.0,1
947,礼物生日,562027.0,1
948,生日礼物礼物,419978.0,38
5,生日礼物,414291.0,803


#### jieba分词、实体识别

In [70]:
used_df['word'].describe()

count     7212
unique    7212
top       送人礼物
freq         1
Name: word, dtype: object

In [109]:
import jieba
import jieba.posseg as pseg
import re

In [123]:
# 停用词
# 创建停用词列表
def get_stopwords_list():
    stopwords = [line.strip() for line in open('stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

# 对句子进行中文分词
def seg_depart(sentence,stopwords_list):
    # 对文档中的每一行进行中文分词
#     sentence_depart = jieba.lcut(sentence.strip())
    # 分词并给出词性
    sentence_depart = pseg.lcut(sentence.strip())
    
    out_list = []
    for word, flag in sentence_depart:
        if word not in stopwords_list:
            if flag in 'a':
                #不保留词性为形容词的词
                continue
            if word != '\t':
                out_list.append(word+flag)

    return ' '.join(out_list)



# 去除停用词
def move_stopwords(sentence_list, stopwords_list):
    # 去停用词
    out_list = []
    for word in sentence_list:
        if word not in stopwords_list:
            if not remove_digits(word):
                continue
            if word != '\t':
                out_list.append(word)
    return ' '.join(out_list)



In [124]:
words = pseg.lcut("我爱北京天安门") #jieba默认模式

print(words[0])
for word, flag in words:
    print('%s %s' % (word, flag))

我/r
我 r
爱 v
北京 ns
天安门 ns


In [130]:
# 分词并去停用词
stopwords = get_stopwords_list()
used_df['word_split'] = used_df['word'].apply(seg_depart, stopwords_list=stopwords)
# used_df['word_split'] = used_df['word_split'].apply(move_stopwords, stopwords_list=stopwords)

used_df.head(60)


,word,lenword_count,searched_day,word_split
937,礼物礼物,4466347.0,1,礼物n 礼物n
3,礼物,4466347.0,761,礼物n
941,的礼物,1339503.0,2,礼物n
942,礼物的,1339503.0,1,礼物n
236,什么礼物,1133560.0,4,礼物n
188,送什么礼物,613979.0,11,送v 礼物n
946,礼物送什么,613979.0,1,礼物n 送v
947,礼物生日,562027.0,1,礼物n 生日t
948,生日礼物礼物,419978.0,38,生日礼物l 礼物n
5,生日礼物,414291.0,803,生日礼物l
